# Pykarambola Demo

This notebook demonstrates how to use **pykarambola** to compute Minkowski functionals and tensors on 3D triangulated surfaces.

## What are Minkowski functionals?

Minkowski functionals are a family of morphological measures that fully characterise the geometry and topology of a body in 3D:

| Functional | Meaning | Rank |
|---|---|---|
| w000 | Volume | scalar |
| w100 | Surface area | scalar |
| w200 | Integrated mean curvature | scalar |
| w300 | Euler characteristic | scalar |
| w010, w110, w210, w310 | Position-weighted vectors | vector |
| w020, w120, w220, w320, w102, w202 | Rank-2 tensors (anisotropy) | 3×3 matrix |

In [1]:
import numpy as np
import pykarambola


def arrays_from_triangulation(tri):
    """Extract vertex/face/label numpy arrays from a Triangulation object."""
    nv, nt = tri.n_vertices(), tri.n_triangles()
    verts = np.array([tri.get_pos_of_vertex(i) for i in range(nv)], dtype=np.float64)
    faces = np.array(
        [[tri.ith_vertex_of_triangle(j, i) for i in range(3)] for j in range(nt)],
        dtype=np.int64,
    )
    labels = np.array([tri.label_of_triangle(j) for j in range(nt)], dtype=np.int64)
    return verts, faces, labels

## 1. Quick start: compute functionals from numpy arrays

The simplest way to use pykarambola is via `minkowski_functionals()`, which takes vertex and face arrays directly.

In [2]:
# Define a simple box (2 x 3 x 4) centred at the origin
a, b, c = 2.0, 3.0, 4.0
ha, hb, hc = a / 2, b / 2, c / 2

verts = np.array([
    [-ha, -hb, -hc],  # 0
    [ ha, -hb, -hc],  # 1
    [ ha,  hb, -hc],  # 2
    [-ha,  hb, -hc],  # 3
    [-ha, -hb,  hc],  # 4
    [ ha, -hb,  hc],  # 5
    [ ha,  hb,  hc],  # 6
    [-ha,  hb,  hc],  # 7
], dtype=np.float64)

# 12 triangles (2 per face), outward-facing normals
faces = np.array([
    [0, 3, 2], [0, 2, 1],  # -z face
    [4, 5, 6], [4, 6, 7],  # +z face
    [0, 1, 5], [0, 5, 4],  # -y face
    [2, 3, 7], [2, 7, 6],  # +y face
    [0, 4, 7], [0, 7, 3],  # -x face
    [1, 2, 6], [1, 6, 5],  # +x face
], dtype=np.int64)

result = pykarambola.minkowski_functionals(verts, faces)

print(f"Volume (w000):               {result['w000']:.4f}  (expected {a*b*c})")
print(f"Surface area (w100):         {result['w100']:.4f}  (expected {2/3*(a*b + b*c + a*c):.4f})")
print(f"Mean curvature (w200):       {result['w200']:.4f}")
print(f"Euler characteristic (w300): {result['w300']:.4f}  (expected {4*np.pi/3:.4f} = 4pi/3)")

Volume (w000):               24.0000  (expected 24.0)
Surface area (w100):         17.3333  (expected 17.3333)
Mean curvature (w200):       9.4248
Euler characteristic (w300): 4.1888  (expected 4.1888 = 4pi/3)


## 2. Loading meshes from files

Pykarambola includes parsers for `.poly` (Geomview), `.off`, `.obj`, and `.glb` files.

In [3]:
# Load a .poly file
tri = pykarambola.parse_poly_file('../test_suite/inputs/box_a=2_b=3_c=4.poly')

print(f"Vertices: {tri.n_vertices()}")
print(f"Triangles: {tri.n_triangles()}")

Vertices: 8
Triangles: 12


In [4]:
# Extract arrays and compute functionals
poly_verts, poly_faces, _ = arrays_from_triangulation(tri)

result = pykarambola.minkowski_functionals(poly_verts, poly_faces)

print("Scalar functionals:")
for name in ['w000', 'w100', 'w200', 'w300']:
    print(f"  {name} = {result[name]:.6f}")

print("\nVector functionals (should be ~0 for a centred box):")
for name in ['w010', 'w110', 'w210', 'w310']:
    print(f"  {name} = {result[name]}")

Scalar functionals:
  w000 = 24.000000
  w100 = 17.333333
  w200 = 9.424778
  w300 = 4.188790

Vector functionals (should be ~0 for a centred box):
  w010 = [0. 0. 0.]
  w110 = [0. 0. 0.]
  w210 = [ 0.00000000e+00  0.00000000e+00 -1.11022302e-16]
  w310 = [0.00000000e+00 0.00000000e+00 2.22044605e-16]


## 3. Rank-2 tensors and eigensystems

Rank-2 functionals capture directional anisotropy. The eigensystem decomposition reveals the principal axes and their magnitudes.

In [5]:
print("Volume tensor (w020):")
print(result['w020'])

print("\nEigenvalues of w020:")
print(result['w020_eigvals'])

print("\nEigenvectors (columns) of w020:")
print(result['w020_eigvecs'])

Volume tensor (w020):
[[ 8.  0.  0.]
 [ 0. 18.  0.]
 [ 0.  0. 32.]]

Eigenvalues of w020:
[ 8. 18. 32.]

Eigenvectors (columns) of w020:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [6]:
# All rank-2 tensor eigenvalues at a glance
print("Rank-2 tensor eigenvalues:")
print(f"{'Name':<8} {'λ₁':>12} {'λ₂':>12} {'λ₃':>12}")
print("-" * 48)
for name in ['w020', 'w120', 'w220', 'w320', 'w102', 'w202']:
    ev = result[f'{name}_eigvals']
    print(f"{name:<8} {ev[0]:12.6f} {ev[1]:12.6f} {ev[2]:12.6f}")

Rank-2 tensor eigenvalues:
Name               λ₁           λ₂           λ₃
------------------------------------------------
w020         8.000000    18.000000    32.000000
w120        11.111111    21.000000    33.777778
w220         8.028515    16.493361    26.529005
w320         4.188790     9.424778    16.755161
w102         4.000000     5.333333     8.000000
w202         2.617994     3.141593     3.665191


## 4. Using an explicit center

Position-dependent functionals (vectors and rank-2 tensors) depend on the choice of origin. Passing an explicit `center` shifts coordinates so the reference point is at the origin, which is useful for isolating shape anisotropy from positional effects.

In [7]:
# Shift the box away from the origin
offset = np.array([10.0, 20.0, 30.0])
shifted_verts = verts + offset

# Without center correction - vectors reflect the offset
result_no_center = pykarambola.minkowski_functionals(shifted_verts, faces)
print("w010 without center correction:", result_no_center['w010'])

# With explicit center correction - vectors reflect only shape
result_centered = pykarambola.minkowski_functionals(
    shifted_verts, faces, center=offset
)
print("w010 with center=offset:       ", result_centered['w010'])

w010 without center correction: [240. 480. 720.]
w010 with center=offset:        [0. 0. 0.]


## 5. Working with labelled surfaces

When your mesh has multiple bodies (e.g., grains, particles), pass a per-face `labels` array. Functionals are computed independently for each label.

In [8]:
# Load a mesh with labels
tri_labelled = pykarambola.parse_poly_file(
    '../test_suite/inputs/two_boxes_sharing_vertex_with_labels.poly',
    with_labels=True,
)

lbl_verts, lbl_faces, lbl_labels = arrays_from_triangulation(tri_labelled)

print(f"Vertices: {tri_labelled.n_vertices()}")
print(f"Triangles: {tri_labelled.n_triangles()}")
print(f"Labels: {sorted(set(int(x) for x in lbl_labels))}")

Vertices: 16
Triangles: 24
Labels: [1, 2]


In [9]:
# Compute per-label functionals
results = pykarambola.minkowski_functionals(
    lbl_verts, lbl_faces, labels=lbl_labels,
)

for label, funcs in sorted(results.items()):
    print(f"\nLabel {label}:")
    print(f"  Volume (w000):       {funcs['w000']:.4f}")
    print(f"  Surface area (w100): {funcs['w100']:.4f}")
    print(f"  Euler char (w300):   {funcs['w300']:.4f}")


Label 1:
  Volume (w000):       -24.0000
  Surface area (w100): 17.3333
  Euler char (w300):   3.1416

Label 2:
  Volume (w000):       -24.0000
  Surface area (w100): 17.3333
  Euler char (w300):   3.1416


/var/folders/v_/7mhfqnhs3d76987qq5hjcz2c0000gn/T/ipykernel_29982/655772026.py:2: UserWarning: Negative volume (w000=-24) detected for label 1. This usually indicates inverted face winding (inward normals).
  results = pykarambola.minkowski_functionals(
/var/folders/v_/7mhfqnhs3d76987qq5hjcz2c0000gn/T/ipykernel_29982/655772026.py:2: UserWarning: Negative volume (w000=-24) detected for label 2. This usually indicates inverted face winding (inward normals).
  results = pykarambola.minkowski_functionals(


## 6. Computing from a 3D label image (voxel data)

If your data is a 3D voxel image (e.g., from a CT scan segmentation), use `minkowski_functionals_from_label_image()`. It internally runs marching cubes to create a triangulated surface.

**Requires:** `scikit-image`

In [10]:
# Create a synthetic 3D label image with a sphere
shape = (64, 64, 64)
z, y, x = np.mgrid[:shape[0], :shape[1], :shape[2]]
center = np.array(shape) / 2
radius = 20.0

dist = np.sqrt((x - center[2])**2 + (y - center[1])**2 + (z - center[0])**2)
label_image = np.zeros(shape, dtype=int)
label_image[dist <= radius] = 1

print(f"Image shape: {label_image.shape}")
print(f"Voxels in sphere: {np.sum(label_image == 1)}")

Image shape: (64, 64, 64)
Voxels in sphere: 33401


In [11]:
results = pykarambola.minkowski_functionals_from_label_image(label_image)

r = results[1]  # label 1
analytical_vol = 4 / 3 * np.pi * radius**3

print(f"Measured volume (w000):  {r['w000']:.1f}")
print(f"Analytical sphere vol:   {analytical_vol:.1f}")
print(f"Relative error:          {abs(r['w000'] - analytical_vol) / analytical_vol:.2%}")
print(f"\nMeasured area (w100):    {r['w100']:.1f}")
print(f"Euler char (w300):       {r['w300']:.4f}")

Measured volume (w000):  33359.5
Analytical sphere vol:   33510.3
Relative error:          0.45%

Measured area (w100):    1817.6
Euler char (w300):       4.1888


Note: The voxelised sphere approximation improves with higher resolution (larger images).

## 7. Selecting specific functionals

For performance, you can request only the functionals you need.

In [12]:
# Only compute volume and surface area
result = pykarambola.minkowski_functionals(
    verts, faces, compute=['w000', 'w100']
)
print("Keys returned:", sorted(result.keys()))
print(f"Volume: {result['w000']:.4f}")
print(f"Area:   {result['w100']:.4f}")

Keys returned: ['w000', 'w100']
Volume: 24.0000
Area:   17.3333


In [13]:
# Compute everything including higher-order tensors and spherical Minkowski
result_all = pykarambola.minkowski_functionals(
    verts, faces, compute='all'
)
print("All keys:", sorted(result_all.keys()))
print(f"\nw103 shape (rank-3 tensor): {result_all['w103'].shape}")
print(f"w104 shape (rank-4 tensor): {result_all['w104'].shape}")
print(f"Spherical Minkowski ql:     {result_all['msm_ql']}")
print(f"Spherical Minkowski wl:     {result_all['msm_wl']}")

All keys: ['msm_ql', 'msm_wl', 'w000', 'w010', 'w020', 'w020_eigvals', 'w020_eigvecs', 'w100', 'w102', 'w102_eigvals', 'w102_eigvecs', 'w103', 'w104', 'w110', 'w120', 'w120_eigvals', 'w120_eigvecs', 'w200', 'w202', 'w202_eigvals', 'w202_eigvecs', 'w210', 'w220', 'w220_eigvals', 'w220_eigvecs', 'w300', 'w310', 'w320', 'w320_eigvals', 'w320_eigvecs']

w103 shape (rank-3 tensor): (3, 3, 3)
w104 shape (rank-4 tensor): (6, 6)
Spherical Minkowski ql:     [1.00000000e+00 5.49296660e-17 2.03519332e-01 1.14167017e-16
 7.74978526e-01 1.91136770e-16 4.01550250e-01 2.64273725e-16
 7.31907009e-01 0.00000000e+00 0.00000000e+00 0.00000000e+00]
Spherical Minkowski wl:     [ 1.          0.         -0.102853    0.          0.41614917  0.
 -0.10613408  0.          0.28658427  0.          0.          0.        ]


## 8. Low-level API: individual functional calculators

For maximum control, you can use the individual `calculate_w*` functions directly on a `Triangulation` object.

In [14]:
from pykarambola import Triangulation, calculate_w000, LABEL_UNASSIGNED

# Build a Triangulation from arrays
surface = Triangulation.from_arrays(verts, faces)

print(f"Vertices: {surface.n_vertices()}")
print(f"Triangles: {surface.n_triangles()}")

# Compute volume using the low-level calculator
raw_w000 = calculate_w000(surface)
# Returns dict[label] -> MinkValResult
for label, mink_val in raw_w000.items():
    print(f"\nLabel {label}: volume = {mink_val.result:.6f}")

Vertices: 8
Triangles: 12

Label 0: volume = 24.000000


## 9. Loading different file formats

In [15]:
# .off format
tri_off = pykarambola.parse_off_file('../test_suite/inputs/cuboid.off')
off_verts, off_faces, _ = arrays_from_triangulation(tri_off)
print(f"OFF: {tri_off.n_vertices()} vertices, {tri_off.n_triangles()} triangles")

result_off = pykarambola.minkowski_functionals(off_verts, off_faces)
print(f"  Volume: {result_off['w000']:.4f}")
print(f"  Area:   {result_off['w100']:.4f}")

OFF: 8 vertices, 12 triangles
  Volume: 4.0000
  Area:   5.1046


In [16]:
# .obj format
tri_obj = pykarambola.parse_obj_file('../test_suite/inputs/box.obj')
obj_verts, obj_faces, _ = arrays_from_triangulation(tri_obj)
print(f"OBJ: {tri_obj.n_vertices()} vertices, {tri_obj.n_triangles()} triangles")

result_obj = pykarambola.minkowski_functionals(obj_verts, obj_faces)
print(f"  Volume: {result_obj['w000']:.4f}")
print(f"  Area:   {result_obj['w100']:.4f}")

OBJ: 8 vertices, 12 triangles
  Volume: 24.0000
  Area:   17.3333


## Summary

| Task | Function |
|---|---|
| Compute from arrays | `minkowski_functionals(verts, faces)` |
| Compute from voxels | `minkowski_functionals_from_label_image(label_image)` |
| Load .poly file | `parse_poly_file(path)` |
| Load .off file | `parse_off_file(path)` |
| Load .obj file | `parse_obj_file(path)` |
| Load .glb file | `parse_glb_file(path)` |
| Per-label analysis | Pass `labels=` array |
| Centroid correction | Pass `center='centroid'` |
| Select functionals | Pass `compute=['w000', 'w100', ...]` |
| All functionals | Pass `compute='all'` |

In [18]:
help(pykarambola.minkowski_functionals)

Help on function minkowski_functionals in module pykarambola.api:

minkowski_functionals(
    verts,
    faces,
    labels=None,
    center=None,
    compute='standard'
)
    Compute Minkowski functionals on a triangulated surface.

    Parameters
    ----------
    verts : (V, 3) array_like
        Vertex positions.
    faces : (F, 3) array_like
        Triangle vertex indices.
    labels : (F,) array_like or None
        Per-face labels. If None, treat as a single body.
    center : None, 'centroid', or (3,) array_like
        Reference point for position-dependent tensors.
        None: use origin. 'centroid': use per-functional centroid.
        (3,) array: shift vertices by -center before computing.
    compute : str or list of str
        'standard' (14 base functionals + eigensystems),
        'all' (adds w103, w104, msm), or list of names.

    Returns
    -------
    dict or dict[int, dict]
        When labels is None: dict mapping functional names to values.
        When labe